# Reading Test Scores

## Problem 1.1 - Dataset size
Load the training and testing sets, and save them as variables with the names pisa_train and pisa_test.

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
pisa_train = pd.read_csv('../data/pisa2009train.csv')
pisa_test = pd.read_csv('../data/pisa2009test.csv')

How many students are there in the training set?

In [3]:
pisa_train.shape[0]

3663

## Problem 1.2 - Summarizing the dataset
What is the average reading test score of males?

In [4]:
pisa_train[pisa_train['male']==1]['readingScore'].mean()

483.53247863247867

Of females?

In [5]:
pisa_train[pisa_train['male']==0]['readingScore'].mean()

512.94063093244

## Problem 1.3 - Locating missing values
Which variables are missing data in at least one observation in the training set? *Select all that apply*.
- raceeth
- preschool
- expectBachelors
- motherHS
- motherBachelors
- motherWork
- fatherHS
- fatherBachelors
- fatherWork
- selfBornUS
- motherBornUS
- fatherBornUS
- englishAtHome
- computerForSchoolWork
- read30MinsADay
- minutesPerWeekEnglish
- studentsInEnglish
- schoolHasLibrary
- schoolSize

In [6]:
pisa_train.isnull().sum()

grade                      0
male                       0
raceeth                   35
preschool                 56
expectBachelors           62
motherHS                  97
motherBachelors          397
motherWork                93
fatherHS                 245
fatherBachelors          569
fatherWork               233
selfBornUS                69
motherBornUS              71
fatherBornUS             113
englishAtHome             71
computerForSchoolwork     65
read30MinsADay            34
minutesPerWeekEnglish    186
studentsInEnglish        249
schoolHasLibrary         143
publicSchool               0
urban                      0
schoolSize               162
readingScore               0
dtype: int64

## Problem 1.4 - Removing missing values
Linear regression discards observations with missing data, so we will remove all such observations from the training and testing sets. Later in the course, we will learn about imputation, which deals with missing data by filling in missing values with plausible information.

Remove observations with any missing value from pisa_train and pisa_test.

How many observations are now in the training set?
- 

How many observations are now in the testing set?
- 

In [7]:
pisa_train.dropna(inplace=True)
pisa_test.dropna(inplace=True)

print("Train set:", pisa_train.shape[0])
print("Test set:", pisa_test.shape[0])

Train set: 2414
Test set: 990


## Problem 2.1 - Factor variables
Which of the following variables is an unordered factor with at least 3 levels? *Select all that apply*.
- raceeth

Which of the following variables is an ordered factor with at least 3 levels? *Select all that apply*.
- grade

In [8]:
for col in ['grade', 'male', 'raceeth']:
    print(pisa_train[col].unique())

[11 10  9 12  8]
[1 0]
['White' 'Black' 'Hispanic' 'More than one race'
 'American Indian/Alaska Native' 'Asian'
 'Native Hawaiian/Other Pacific Islander']


## Problem 2.2 - Unordered factors in regression models
To include unordered factors in a linear regression model, we define one level as the "reference level" and add a binary variable for each of the remaining levels. In this way, a factor with n levels is replaced by n-1 binary variables. The reference level is typically selected to be the most frequently occurring level in the dataset.

As an example, consider the unordered factor variable "color", with levels "red", "green", and "blue". If "green" were the reference level, then we would add binary variables "colorred" and "colorblue" to a linear regression problem. All red examples would have colorred=1 and colorblue=0. All blue examples would have colorred=0 and colorblue=1. All green examples would have colorred=0 and colorblue=0.

Now, consider the variable "raceeth" in our problem, which has levels "American Indian/Alaska Native", "Asian", "Black", "Hispanic", "More than one race", "Native Hawaiian/Other Pacific Islander", and "White". Because it is the most common in our population, we will select White as the reference level.

Which binary variables will be included in the regression model? *Select all that apply*.
- raceethAmerican Indian/Alaska Native
- raceethAsian
- raceethBlack
- raceethHispanic
- raceethMore than one race
- raceethNative Hawaiian/Other Pacific Islander

## Problem 2.3 - Example unordered factors
Consider again adding our unordered factor race to the regression model with reference level "White".

For a student who is Asian, which binary variables would be set to 0? All remaining variables will be set to 1. *Select all that apply*.
- raceethAmerican Indian/Alaska Native
- raceethBlack
- raceethHispanic
- raceethMore than one race
- raceethNative Hawaiian/Other Pacific Islander

For a student who is white, which binary variables would be set to 0? All remaining variables will be set to 1. *Select all that apply*.
- raceethAmerican Indian/Alaska Native
- raceethAsian
- raceethBlack
- raceethHispanic
- raceethMore than one race
- raceethNative Hawaiian/Other Pacific Islander

## Problem 3.1 - Building a model
Now, build a linear regression model (call it lm_score) using the training set to predict readingScore using all the remaining variables.

What is the Multiple R-squared value of lm_score on the training set?
- 0.325

In [9]:
raceeth_train = pd.get_dummies(pisa_train['raceeth'], prefix='raceeth')
pisa_train = pd.merge(pisa_train, raceeth_train, how='left', left_index=True, right_index=True)
raceeth_test = pd.get_dummies(pisa_test['raceeth'], prefix='raceeth')
pisa_test = pd.merge(pisa_test, raceeth_test, how='left', left_index=True, right_index=True)

features = [
    'grade', 'male', 'preschool', 'expectBachelors', 
    'motherHS', 'motherBachelors', 'motherWork', 
    'fatherHS', 'fatherBachelors', 'fatherWork', 
    'selfBornUS', 'motherBornUS', 'fatherBornUS',
    'englishAtHome', 'computerForSchoolwork', 'read30MinsADay',
    'minutesPerWeekEnglish', 'studentsInEnglish', 'schoolHasLibrary',
    'publicSchool', 'urban', 'schoolSize',
    'raceeth_American Indian/Alaska Native', 'raceeth_Asian',
    'raceeth_Black', 'raceeth_Hispanic', 'raceeth_More than one race',
    'raceeth_Native Hawaiian/Other Pacific Islander'
]
X_train = pisa_train[features]
y_train = pisa_train['readingScore']

lm_score = sm.OLS(y_train, sm.add_constant(X_train)).fit()
print(lm_score.summary())

                            OLS Regression Results                            
Dep. Variable:           readingScore   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     41.04
Date:                Sun, 15 Aug 2021   Prob (F-statistic):          1.72e-180
Time:                        08:16:24   Log-Likelihood:                -13795.
No. Observations:                2414   AIC:                         2.765e+04
Df Residuals:                    2385   BIC:                         2.781e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

### Problem 3.2 - Computing the root-mean squared error of the model
What is the training-set root-mean squared error (RMSE) of lm_score?

In [10]:
train_y_pred = lm_score.predict(sm.add_constant(X_train))
train_residue = y_train - train_y_pred
train_sse = (train_residue**2).sum()

rmse_train = np.sqrt(train_sse / X_train.shape[0]).round(3)
print("RMSE for train set", rmse_train)

RMSE for train set 73.366


## Problem 3.3 - Comparing predictions for similar students
Consider two students A and B. They have all variable values the same, except that student A is in grade 11 and student B is in grade 9. What is the predicted reading score of student A minus the predicted reading score of student B?
- 59.09 *(2 * coefficient for grade)*

## Problem 3.4 - Interpreting model coefficients
What is the meaning of the coefficient associated with variable raceethAsian?
- Predicted difference in the reading score between an Asian student and a white student who is otherwise identical.

## Problem 3.5 - Identifying variables lacking statistical significance
Based on the significance codes, which variables are candidates for removal from the model? *Select all that apply*. (We'll assume that the factor variable raceeth should only be removed if none of its levels are significant.)
- preschool
- motherHS
- motherWork
- fatherHS
- fatherWork
- selfBornUS
- motherBornUS
- fatherBornUS
- englishAtHome
- minutesPerWeekEnglish
- studentsInEnglish
- schoolHasLibrary
- urban

## Problem 4.1 - Predicting on unseen data
Use the lm_score model to predict the reading scores of students in pisa_t
est. Call this vector of predictions "pred_test". Do not change the variables in the model (for example, do not remove variables that we found were not significant in the previous part of this problem). Use the summary function to describe the test set predictions.

What is the range between the maximum and minimum predicted reading score on the test set?

In [11]:
X_test = pisa_test[features]
y_test = pisa_test['readingScore']

pred_test = lm_score.predict(sm.add_constant(X_test))
pred_min = pred_test.min().round(3)
pred_max = pred_test.max().round(3)

print(f"Range: {pred_max - pred_min}")

Range: 284.468


## Problem 4.2 - Test set SSE and RMSE
What is the sum of squared errors (SSE) of lm_score on the testing set?

What is the root-mean squared error (RMSE) of lm_score on the testing set?


In [12]:
test_residue = y_test - pred_test
test_sse = (test_residue**2).sum().round(3)
test_rmse = np.sqrt(test_sse / pisa_test.shape[0]).round(3)

print(f"Test SSEL: {test_sse}\nTest RMSE: {test_rmse}")

Test SSEL: 5762082.371
Test RMSE: 76.291


## Problem 4.3 - Baseline prediction and test-set SSE
What is the predicted test score used in the baseline model? Remember to compute this value using the training set and not the test set.
- 517.963

What is the sum of squared errors of the baseline model on the testing set? *HINT: We call the sum of squared errors for the baseline model the total sum of squares (SST)*.
- 7802354.078

In [13]:
mean_score = pisa_train['readingScore'].mean().round(3)

baseline_residue = y_test - y_train.mean()
sst = (baseline_residue**2).sum().round(3)

print(f"Baseline score: {mean_score}\nSST: {sst}")

Baseline score: 517.963
SST: 7802354.078


## Problem 4.4 - Test-set R-squared
What is the test-set R-squared value of lm_score?


In [14]:
test_r2 = (1 - test_sse / sst).round(3)
print("Test R2:", test_r2)

Test R2: 0.261
